# 数值运算
本章开始将运算和函数，记住所有的操作的基本都是对每一列分别进行的，极少有对DataFrame整体或每个元素分别进行变换的函数。
- 数值运算毫无疑问只对数值列起作用。

    @auther: sunzhenhang
    @zhihu: https://www.zhihu.com/people/HANGZS/activities

In [1]:
!cd

E:\ML\实战\pandas实用教程 - 副本


In [2]:
import numpy as np
import pandas as pd

----
# 1. 逐元素二元运算
记住：
- 此类运算都是默认以**label对齐**的；
- 可以认为相加的两个操作数都是先将index和columns拓展至相同后，再执行运算。

In [3]:
df1 = pd.DataFrame([[1,2],[3,4]], index =['a','b'],columns = ['A','B'])
df1

,A,B
a,1,2
b,3,4


In [4]:
df2 = pd.DataFrame([[3,4],[5,6]], index =['b','c'],columns = ['B','C'])
df2

,B,C
b,3,4
c,5,6


## 1.1 `+`

In [5]:
df1 + df2   # df1的'B','b'和df2的'B','b'对上了，其他位置都至少有一个缺失值，此时填NaN

,A,B,C
a,NaN,NaN,NaN
b,NaN,7.0,NaN
c,NaN,NaN,NaN


In [6]:
s1 = pd.Series([1,2], index =['A','B'], name = 'A')
s1

A    1
B    2
Name: A, dtype: int64

In [7]:
df1 + s1  # +号默认将Series的index与DataFrame的columns对齐，然后以DataFrame的index为index，纵向复制构造一个DataFrame。
# 也即s1被转化成了：
#   A	B
#a	1	2
#b	1	2

,A,B
a,2,4
b,4,6


## 1.2  ` .add()`
add比+号功能更强，区别提体现在可以控制Series构造的方式。
#### `DataFrame.add(other, axis='columns', fill_value=None)`
- other：另一个DataFrame或Series；
- axis：如果other是Series，指定Series的索引去和DataFrame的行匹配，还是和列匹配（+号是默认和列匹配）；
- fill_value：这个参数是指两个DataFrame只有一个值缺失的情况，缺失的值怎么处理。两者都缺失，那么就是NaN。

In [8]:
df1.add(s1, axis = 'columns')  # 和+号功能相同

,A,B
a,2,4
b,4,6


In [9]:
s2 = pd.Series([1,2], index =['a','B'])
s2

a    1
B    2
dtype: int64

In [10]:
df1.add(s2, axis = 'index') 
# s1被转化为：以DataFrame的columns为columns，横向复制
#	A	B
#A	1	1
#B	2	2

,A,B
B,NaN,NaN
a,2.0,3.0
b,NaN,NaN


## 1.3 `-, ×, /, //,%, **`  
相应的函数形式：`sub(), mul(), div(), floordiv(), mod(), pow()`。</br>
**这几个操作和`+` 以及`add() `完全相同。**

## 1.4 `divmod()`
divmod()是python内建函数，不是pandas API，其支持Series进行逐元素操作。

In [11]:
d,r = divmod(s1,2)
d

A    0
B    1
Name: A, dtype: int64

In [12]:
r

A    1
B    0
Name: A, dtype: int64

---
# 2. 矩阵运算
pandas是以numpy这个数值代数库为基础的，所以其也支持矩阵操作，虽然它并以此为目的。

## 2.1 `.dot()`
#### `DataFrame.dot(other)`
- other：DataFrame或Series

注意，pandas在做矩阵运算时，**要求：**矩阵的维度相同，左操作数的列索引和右操作数的行索引相同。

In [13]:
df1

,A,B
a,1,2
b,3,4


In [14]:
df2

,B,C
b,3,4
c,5,6


In [15]:
# df1.dot(df2)#报错
df2.index = ['A','B']  
df2

,B,C
A,3,4
B,5,6


In [16]:
df1.dot(df2)  # 左操作数的行索引和右操作数的列索引变为内积后元素的行、列索引

,B,C
a,13,16
b,29,36


In [17]:
s1

A    1
B    2
Name: A, dtype: int64

In [18]:
df1.dot(s1)

a     5
b    11
dtype: int64

## 2.2  `.T`
矩阵的转置。

In [19]:
df1.T

,a,b
A,1,3
B,2,4


--- 
# 3. 本地一元变换
这些运算不改变索引，只是在当前位置做一些变化。

## 3.1 `.abs()`
转化为绝对值。

In [20]:
df1.loc['a','A'] = -1
df1

,A,B
a,-1,2
b,3,4


In [21]:
df1.abs()

,A,B
a,1,2
b,3,4


## 3.2 `.cumxxx()`
累计运算，从开始到当前数据结束获取一个值。
#### `DataFrame.cumxxx(axis='index')`
- axis：'index'或'columns'

xxx 可取：max, min, sum, prod

In [22]:
df1.cummax()  # 默认为沿index，列

,A,B
a,-1,2
b,3,4


In [23]:
df1.cummax(axis = 'columns')  # columns意思，行

,A,B
a,-1,2
b,3,4


## 3.3 `.clip()`
顾名思义，裁剪也即将数据裁剪到一定范围内。
#### `DataFrame.clip(lower=None, upper=None)`
- lower：float 
- upper：float 

In [24]:
df1 = pd.DataFrame([[-1,1],[-2,3]])
df1

,0,1
0,-1,1
1,-2,3


In [25]:
df1.clip( -0.5,0.5)

,0,1
0,-0.5,0.5
1,-0.5,0.5


## 3.4 `.rank()`
顾名思义，rank次序的意思，也即标示出数值的次序
#### `DataFrame.rank(axis=0, method='average',  ascending=True, pct=False)`
- axis：0（'index'）-按列方向，1('columns')-按行方向
- method：对于值相同的数怎么标记顺序，
   - 'first'：所有数按照大小排序，如相同按照出现顺序赋次序。
   - 'dense'：值相同为一组，按照组值排序，组内取相同次序，和min不同—min是将相同数用第一
- ascending：True or False，升序还是降序；
- pct：percent，按照百分比显示，将显示的次序除以列、或行元素个数。

In [26]:
df1 = pd.DataFrame(data = [[1,1,2,2],[2,3,2,4]])
df1

,0,1,2,3
0,1,1,2,2
1,2,3,2,4


In [27]:
df1.rank( axis = 'columns',method = 'min',pct= False) # 注意和下面 dense 对比

,0,1,2,3
0,1.0,1.0,3.0,3.0
1,1.0,3.0,1.0,4.0


In [28]:
df1.rank( axis = 'columns',method = 'dense',pct= False)  #

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,1.0,2.0,1.0,3.0


In [29]:
df1.rank( axis = 'columns',method = 'dense',pct= True)  # 按照百分比显示，相当于直接除行元素个数

,0,1,2,3
0,0.25,0.25,0.50,0.50
1,0.25,0.50,0.25,0.75


## 3.5 `.round()`
前面一章介绍了设置显示精度的操作，其不改变实际精度，而这个API则可以改变精度。

#### `Series.round(decimals=0)`
- decimals：控制小数位数

In [30]:
s= pd.Series([1.111,2.222])
s

0    1.111
1    2.222
dtype: float64

In [31]:
s.round(1)

0    1.1
1    2.2
dtype: float64

In [32]:
s.round(2)

0    1.11
1    2.22
dtype: float64